In [2]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

df = pd.read_csv(list(uploaded.keys())[0])

print(df.head())


Saving AWCustomers.csv to AWCustomers.csv
   CustomerID Title FirstName MiddleName  LastName Suffix  \
0       21173   NaN      Chad          C      Yuan    NaN   
1       13249   NaN      Ryan        NaN     Perry    NaN   
2       29350   NaN     Julia        NaN  Thompson    NaN   
3       13503   NaN  Theodore        NaN     Gomez    NaN   
4       22803   NaN  Marshall          J      Shan    NaN   

             AddressLine1 AddressLine2         City    StateProvinceName  ...  \
0      7090 C. Mount Hood          NaN   Wollongong      New South Wales  ...   
1     3651 Willow Lake Rd          NaN      Shawnee     British Columbia  ...   
2  1774 Tice Valley Blvd.          NaN  West Covina           California  ...   
3         2103 Baldwin Dr          NaN    Liverpool              England  ...   
4         Am Gallberg 234          NaN        Werne  Nordrhein-Westfalen  ...   

         Education      Occupation Gender MaritalStatus HomeOwnerFlag  \
0        Bachelors        Cleri

In [3]:
print(df.columns)

columns_to_drop = ['Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix', 'AddressLine1', 'AddressLine2', 'LastUpdated']

df_cleaned = df.drop(columns=columns_to_drop)

print(df_cleaned.columns)


Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome', 'LastUpdated'],
      dtype='object')
Index(['CustomerID', 'City', 'StateProvinceName', 'CountryRegionName',
       'PostalCode', 'PhoneNumber', 'BirthDate', 'Education', 'Occupation',
       'Gender', 'MaritalStatus', 'HomeOwnerFlag', 'NumberCarsOwned',
       'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome'],
      dtype='object')


In [13]:
df_cleaned.isnull().sum()

,0
CustomerID,0
City,0
StateProvinceName,0
CountryRegionName,0
PostalCode,0
PhoneNumber,0
BirthDate,0
Education,0
Occupation,0
Gender,0


In [12]:
data1 = [101, "Seattle", "Washington", "USA", "98101", "206-555-1234", "1985-07-15", "Bachelor's Degree", "Software Engineer", "M", None, 1, 2, None, None, 95000]
data2 = [102, "Austin", "Texas", "USA", "73301", "512-555-5678", "1990-03-22", "Master's Degree", "Marketing Manager", "F", None, 0, 1, None, None, 75000]
data3 = [103, "Chicago", "Illinois", "USA", "60601", "312-555-9012", "1978-11-05", "High School", "Electrician", "M", None, 1, 3, None, None, 60000]
data4 = [104, "Denver", "Colorado", "USA", "80202", "303-555-3456", "1988-06-30", "Associate Degree", "Graphic Designer", "F", None, 0, 1, None, None, 55000]
data5 = [105, "Miami", "Florida", "USA", "33101", "305-555-7890", "1995-12-12", "Bachelor's Degree", "Teacher", "F", None, 1, 0, None, None, 48000]
columns = [
    "CustomerID", "City", "StateProvinceName", "CountryRegionName", "PostalCode", "PhoneNumber", "BirthDate",
    "Education", "Occupation", "Gender", "MaritalStatus", "HomeOwnerFlag",
    "NumberCarsOwned", "NumberChildrenAtHome", "TotalChildren", "YearlyIncome"
]
data = [data1, data2, data3, data4, data5]
df_new = pd.DataFrame(data, columns=columns)
df_cleaned = pd.concat([df_cleaned, df_new], ignore_index=True)
print(df_cleaned.tail())

       CustomerID     City StateProvinceName CountryRegionName PostalCode  \
18371         101  Seattle        Washington               USA      98101   
18372         102   Austin             Texas               USA      73301   
18373         103  Chicago          Illinois               USA      60601   
18374         104   Denver          Colorado               USA      80202   
18375         105    Miami           Florida               USA      33101   

        PhoneNumber   BirthDate          Education         Occupation Gender  \
18371  206-555-1234  1985-07-15  Bachelor's Degree  Software Engineer      M   
18372  512-555-5678  1990-03-22    Master's Degree  Marketing Manager      F   
18373  312-555-9012  1978-11-05        High School        Electrician      M   
18374  303-555-3456  1988-06-30   Associate Degree   Graphic Designer      F   
18375  305-555-7890  1995-12-12  Bachelor's Degree            Teacher      F   

      MaritalStatus  HomeOwnerFlag  NumberCarsOwned Numb

In [32]:
df_cleaned = df_cleaned.fillna(df_cleaned.median(numeric_only=True))
df_cleaned = df_cleaned.fillna(df_cleaned.mode().iloc[0])

In [33]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, KBinsDiscretizer, OneHotEncoder
scaler = MinMaxScaler()
df_cleaned[["YearlyIncome"]] = scaler.fit_transform(df_cleaned[["YearlyIncome"]])

In [35]:
binning = KBinsDiscretizer(n_bins=3, encode="ordinal", strategy="uniform")
df_cleaned["NumberChildrenAtHome"] = binning.fit_transform(df_cleaned[["NumberChildrenAtHome"]])

In [36]:
std_scaler = StandardScaler()
df_cleaned[["NumberCarsOwned"]] = std_scaler.fit_transform(df_cleaned[["NumberCarsOwned"]])

In [37]:
cat_cols = ["Gender", "Education", "Occupation", "NumberChildrenAtHome"]
df_encoded = pd.get_dummies(df_cleaned, columns=cat_cols, drop_first=True)

In [40]:
cat_cols = ["City", "StateProvinceName", "CountryRegionName", "PostalCode",
            "PhoneNumber", "BirthDate", "Education", "Occupation",
            "Gender", "MaritalStatus"]
num_cols = ["CustomerID", "HomeOwnerFlag", "NumberCarsOwned",
            "NumberChildrenAtHome", "TotalChildren", "YearlyIncome"]

In [44]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoded = encoder.fit_transform(df[cat_cols])
scaler = StandardScaler()
scaled = scaler.fit_transform(df[num_cols])
df_encoded = pd.DataFrame(
    np.hstack([scaled, encoded]),
    columns = list(num_cols) + list(encoder.get_feature_names_out(cat_cols))
)

In [45]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import jaccard
from scipy.stats import pearsonr
import numpy as np
obj1 = df_encoded.iloc[0].values.reshape(1, -1)
obj2 = df_encoded.iloc[1].values.reshape(1, -1)

In [46]:
# Cosine similarity
cos_sim = cosine_similarity(obj1, obj2)[0][0]

# Jaccard similarity (for binary attributes only)
jacc_sim = 1 - jaccard(obj1[0] > 0, obj2[0] > 0)

# Simple Matching Coefficient (SMC)
smc = np.sum(obj1 == obj2) / len(obj1[0])

print("Cosine Similarity:", cos_sim)
print("Jaccard Similarity:", jacc_sim)
print("Simple Matching Coefficient:", smc)

# Correlation between Commute Distance & Yearly Income
corr, pval = pearsonr(df["NumberChildrenAtHome"], df["YearlyIncome"])
print("Correlation:", corr, " (p-value:", pval, ")")

Cosine Similarity: 0.29306667182336965
Jaccard Similarity: 0.30434782608695654
Simple Matching Coefficient: 0.9989303608624669
Correlation: 0.00474603073679213  (p-value: 0.5200714614552047 )
